In [0]:
%pip install openai redlines
dbutils.library.restartPython()

In [0]:
import os
from openai import OpenAI 
from pydantic import BaseModel
from IPython.display import display, HTML

In [0]:
dbutils.widgets.text("workspace_url", "")
dbutils.widgets.text("base_llm_model", "databricks-meta-llama-3-3-70b-instruct")
ws_url = dbutils.widgets.get("workspace_url")
base_url = f"{ws_url.rstrip('/')}//serving-endpoints"
base_llm_model = dbutils.widgets.get("base_llm_model")
embedding_model = "databricks-gte-large-en"

In [0]:
base_llm_model

In [0]:
#Env Setup for OpenAI token
DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

client = OpenAI(
  api_key=DATABRICKS_TOKEN,
  base_url=base_url
)

In [0]:
# --------------------------------------------------------------
# Step 1: Define the response format in a Pydantic model
# --------------------------------------------------------------

class CalendarEvent(BaseModel):
  name: str
  date: str
  participants: list[str]

In [0]:
# --------------------------------------------------------------
# Step 2: Call the model
# --------------------------------------------------------------

completion = client.beta.chat.completions.parse(
    model=base_llm_model,
    messages = [
        {"role": "system", "content": "Extract the event information."},
        {
            "role": "user",
            "content": "Alice and Bob are going to a science fair on Friday.",
        },
    ],
    response_format=CalendarEvent,
)

In [ ]:
# --------------------------------------------------------------
# Step 3: Parse the response
# --------------------------------------------------------------

event = completion.choices[0].message.parsed

print(f"Event Name: {event.name}")
print(f"Event Date: {event.date}")
print(f"Event Participants: {', '.join(event.participants)}")
# --------------------------------------------------------------